In [23]:
import pygame, sys
from pygame.locals import *
import numpy as np
import tensorflow as tf
from keras.models import load_model
import cv2

White = (255, 255, 255)
Black = (0, 0, 0)
Red = (255, 0, 0)
ImageSave = False
Boundary = 5
image_count = 1
Predict = True

model = tf.keras.models.load_model('bestmodelNEW.h5')

Labels = {0:"Zero", 1:"One", 2:"Two", 3:"Three", 4:"Four", 5:"Five", 6:"Six", 7:"Seven", 8:"Eight", 9:"Nine"}

pygame.init()  
FONT = pygame.font.Font(None, 36)
screen = pygame.display.set_mode((400,500))
pygame.display.set_caption("Digital Display")

iswriting = False
number_xcord = []
number_ycord = []

while True:  
    for event in pygame.event.get():  
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
            
        if event.type == MOUSEMOTION and iswriting:
            xcord, ycord = event.pos     #x, y mouse position
            pygame.draw.circle(screen, White, (xcord, ycord), 4, 0)     # display surface, 4=thickness, 0=brightness

            number_xcord.append(xcord)
            number_ycord.append(ycord)
        
        if event.type == MOUSEBUTTONDOWN:
            iswriting = True
        
        if event.type == MOUSEBUTTONUP:
            iswriting = False
            number_xcord = sorted(number_xcord)
            number_ycord = sorted(number_ycord)
            
            rectancular_min_x, rectancular_max_x = max(number_xcord[0] - Boundary, 0), min(number_xcord[-1] + Boundary, screen.get_width())
            rectancular_min_y, rectancular_max_y = max(number_ycord[0] - Boundary, 0), min(number_ycord[-1] + Boundary, screen.get_height())
            
            # Extract the region of interest (ROI) from the screen
            img_arr = np.array(pygame.PixelArray(screen))[rectancular_min_x:rectancular_max_x, rectancular_min_y:rectancular_max_y].T.astype(np.float32) # transpose correct orientation, float
            
            if ImageSave: # save the image
                cv2.imwrite("image.png", img_arr)  # pass the value, file name
                image_count += 1
            
            if Predict:
                image = cv2.resize(img_arr, (28, 28)) # because model trained 28, 28j
                image = np.pad(image, (10, 10), 'constant', constant_values = 0)
                image = cv2.resize(image, (28, 28))/255 # Normalize
                
                label = str(Labels[np.argmax(model.predict(image.reshape(1,28,28,1)))]) # argmax returns string
                
                textSurface = FONT.render(label, True, Red, White)
                textRectancularObject = textSurface.get_rect() # get rectangular
                textRectancularObject.left, textRectancularObject.bottom = rectancular_min_x, rectancular_max_y
                
                screen.blit(textSurface, textRectancularObject) # blit use to display my object
            
            # Reset coordinates
            number_xcord.clear()
            number_ycord.clear()
        
        pygame.display.update()


1/1 [==============================] - 0s 23ms/step


SystemExit: 